In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [13]:
URL = "https://www.redbus.in/online-booking/north-bengal-state-transport-corporation"
# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)    # Wait for the page to load

In [14]:
# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('northbengal.csv', index=False)

# Close the driver
driver.quit()

In [15]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NORTH BENGAL STATE TRANSPORT CORPORATION,Non AC Seater (2+3),15:00,13h 20m,04:20,3.7,INR 450,49 Seats available
1,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-COB-R1-17:00,Non AC Seater 2+2,17:00,12h 00m,05:00,3.6,INR 486,25 Seats available
2,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-SLG-R1-18:00,NON A/C Seater Push Back (2+2),18:00,12h 00m,06:00,4.0,INR 486,18 Seats available
3,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,EASY RIDE,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:45,12h 40m,07:25,4.7,INR 1500,14 Seats available
4,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Shyamoli Paribahan Pvt Ltd,Volvo Multi-Axle A/C seater/sleeper (2+1),18:30,13h 15m,07:45,4.6,1200,41 Seats available
...,...,...,...,...,...,...,...,...,...,...
354,Raiganj to Ranaghat,https://www.redbus.in/bus-tickets/raiganj-to-r...,Lokenath Bus Service,NON A/C Seater / Sleeper (2+2),19:45,08h 25m,04:10,2.6,INR 500,12 Seats available
355,Raiganj to Ranaghat,https://www.redbus.in/bus-tickets/raiganj-to-r...,Aradhana Bus Service,NON A/C Seater / Sleeper (2+2),19:15,07h 55m,03:10,2.4,665,33 Seats available
356,Cooch Behar (West Bengal) to Farakka,https://www.redbus.in/bus-tickets/cooch-behar-...,NBSTC-COB-BHP-R1-19:00,Non AC Seater 2+2,19:00,09h 30m,04:30,4.5,INR 367,7 Seats available
357,Cooch Behar (West Bengal) to Siliguri,https://www.redbus.in/bus-tickets/cooch-behar-...,NBSTC-COB-FKT-SLG-08-10:25,Non AC Seater (2+3),10:25,04h 00m,14:25,2.8,INR 119,46 Seats available
